In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint
from sklearn.metrics import mean_absolute_error
import pandas as pd
import mlflow
import os
import numpy as np

In [23]:
"""
Descarga de Archivos desde Google Drive sin Autenticación

Este script permite descargar archivos específicos desde Google Drive directamente a una ubicación local,
utilizando el ID de Google Drive del archivo (el archivo debe ser accesible para cualquier persona que tenga la URL).

El script procesa un archivo de texto ('archivos_info.txt') que contiene las IDs de Google Drive de los archivos,
los nombres con los que se desean guardar localmente, y las rutas locales de almacenamiento, todo separado por comas.

Formato esperado de 'archivos_info.txt':
ID_de_Google_Drive,Nombre_Archivo_Local,Ruta_Local

Ejemplo:
1hGvKmNAkK...,mi_archivo.txt,./descargas

Requisitos:
- Módulo 'requests' instalado en el entorno Python donde se ejecute este script.



"""

import requests
import os


def descargar_archivo_directo(id_archivo, directorio_destino, archivo_destino):
    """
    Descarga un archivo directamente desde Google Drive y lo guarda localmente.

    Parámetros:
    - id_archivo (str): ID del archivo en Google Drive.
    - directorio_destino (str): Ruta del directorio local donde se guardará el archivo.
    - archivo_destino (str): Nombre deseado para el archivo en local

    Devuelve:
    - archivo_destino (str): Nombre del archivo guardado.
    - ruta_completa (str): Ruta completa del archivo guardado.
    """
    # Construye la URL de descarga directa utilizando el ID del archivo
    url = f"https://drive.google.com/uc?export=download&id={id_archivo}"

    # Realiza la petición HTTP GET para descargar el archivo
    respuesta = requests.get(url, allow_redirects=True)

    # Comprueba que el directorio destino existe, si no, lo crea
    os.makedirs(directorio_destino, exist_ok=True)

    # Construye la ruta completa donde se guardará el archivo en local
    ruta_completa = os.path.join(directorio_destino, archivo_destino)

    # Guarda el contenido del archivo descargado en local
    with open(ruta_completa, 'wb') as archivo:
        archivo.write(respuesta.content)


    return archivo_destino, ruta_completa


def procesar_archivo_info(ruta_archivo_info):
    """
    Procesa un archivo de texto que contiene información sobre los archivos a descargar.

    Parámetros:
    - ruta_archivo_info (str): Ruta del archivo de texto que contiene los IDs de Google Drive,
                               los nombres de los archivos locales y las rutas locales.

    Devuelve:
    - Una lista de tuplas con el ID de Google Drive, el nombre local del archivo, y la ruta local.
    """
    archivos_info = []
    with open(ruta_archivo_info, 'r') as archivo:
        for linea in archivo:
            id_archivo, nombre_archivo, directorio_destino = linea.strip().split(',')
            archivos_info.append((id_archivo, nombre_archivo, directorio_destino))
    return archivos_info


In [24]:
# Ruta al archivo que contiene la información de los archivos a descargar
ruta_archivo_info = "info_archivos_GDrive_solo_entr_pos.txt"

# Obtenemos la lista con la info de los archivos del fichero
archivos_a_descargar = procesar_archivo_info(ruta_archivo_info)

# Descargamos cada archivo de la lista
for id_archivo, nombre_archivo, directorio_destino in archivos_a_descargar:
    nombre_archivo_descargado, ruta_archivo_guardado = descargar_archivo_directo(id_archivo, directorio_destino, nombre_archivo)
    print(f"Archivo {nombre_archivo_descargado} guardado en: {ruta_archivo_guardado}")

Archivo datos_preparados_solo_entr_pos.parquet guardado en: ../Downloads\datos_preparados_solo_entr_pos.parquet


In [25]:
df = pd.read_parquet(ruta_archivo_guardado, engine = "pyarrow")

df.describe()

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [16]:
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]

# Partición de datos
RANDOM_STATE = 83
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_STATE)

In [17]:
# Establecer la URI de la base de datos SQLite
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'

# Configuración de MLflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("arboles")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1713174288421, experiment_id='1', last_update_time=1713174288421, lifecycle_stage='active', name='arboles', tags={}>

In [18]:
# Mapeo de posiciones a dos grupos solamente: Porteros y Otros
def map_position_to_group(position):
    if position in ['Centre-Back', 'Left-Back', 'Right-Back']:
        return 'Defensas'
    elif position == 'Goalkeeper':
        return 'Porteros'
    elif position in ['Centre-Forward', 'Second Striker']:
        return 'Delanteros'
    else:
        return 'Medios'

# Crear una nueva columna 'Pos_Group' para grupos de posiciones
df['Pos_Group'] = df['position'].apply(map_position_to_group)

# Eliminar la columna 'position' ahora que tenemos 'Pos_Group'
df.drop(columns=['position'], inplace=True)

# Separar por grupos de posiciones y entrenar un modelo para cada uno
for pos_group in ['Porteros', 'Defensas', 'Medios', 'Delanteros']:  # Solo usamos los grupos definidos
    # Crear un DataFrame para el grupo de posiciones actual
    df_group = df[df['Pos_Group'] == pos_group].copy()

    # Eliminar la columna 'Pos_Group' del DataFrame actual
    df_group.drop(columns=['Pos_Group'], inplace=True)

    X = df_group.iloc[:, :-1]
    Y = df_group.iloc[:, -1]

    # Partición de datos
    RANDOM_STATE = 83
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_STATE)

    # Crear el estimador
    clf = DecisionTreeRegressor(random_state=RANDOM_STATE)

    # Realizar validación cruzada
    cv_scores = cross_validate(clf, X_train, y_train, scoring='neg_mean_absolute_error', cv=10, n_jobs=-1)

    # Entrenar el modelo final
    clf.fit(X_train, y_train)

    # Predecir con el modelo entrenado
    y_pred = clf.predict(X_test)

    # Calcular el error en el conjunto de prueba
    test_error = mean_absolute_error(y_test, y_pred)

    # Iniciar una nueva run de MLflow
    with mlflow.start_run(run_name=f"prueba_{pos_group}"):
        # Registrar los hiperparámetros
        mlflow.log_param("random_state", RANDOM_STATE)

        # Registrar el error en el conjunto de prueba
        mlflow.log_metric(f"test_error_{pos_group}", test_error)

        # Guardar el modelo entrenado
        mlflow.sklearn.log_model(clf, f"model_{pos_group}")

C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Ar